In [31]:
import os
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split # THAY ĐỔI: Dùng train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.transforms import v2 as T
from tqdm.notebook import tqdm


In [32]:
import warnings # <<< MỚI: Thêm thư viện warnings

# <<< MỚI: Tắt các cảnh báo FutureWarning >>>
warnings.simplefilter(action='ignore', category=FutureWarning)

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

Using device: cuda


In [34]:
class TimeShift(torch.nn.Module):
    """ Dịch chuyển spectrogram theo trục thời gian một cách ngẫu nhiên. """
    def __init__(self, p=0.5, max_shift_percent=0.4):
        super().__init__()
        self.p = p
        self.max_shift_percent = max_shift_percent

    def forward(self, spec):
        if torch.rand(1) < self.p:
            time_steps = spec.shape[-1]
            # Chọn một lượng dịch chuyển ngẫu nhiên
            shift_amount = int(torch.rand(1) * self.max_shift_percent * time_steps)
            # Dịch ngẫu nhiên sang trái hoặc phải
            if torch.rand(1) < 0.5:
                shift_amount = -shift_amount
            return torch.roll(spec, shifts=shift_amount, dims=-1)
        return spec

class SpecNoise(torch.nn.Module):
    """ Thêm nhiễu Gaussian trực tiếp vào spectrogram. """
    def __init__(self, p=0.5, noise_level=0.05):
        super().__init__()
        self.p = p
        self.noise_level = noise_level

    def forward(self, spec):
        if torch.rand(1) < self.p:
            noise = torch.randn_like(spec) * self.noise_level
            return spec + noise
        return spec

train_transform = T.Compose([
    # Kỹ thuật 1: SpecAugment (Time & Frequency Masking)
    torchaudio.transforms.TimeMasking(time_mask_param=80),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=25),
    
    # Kỹ thuật 2: Time Shifting
    TimeShift(p=0.5, max_shift_percent=0.4),
    
    # Kỹ thuật 3: Spectrogram Noise
    SpecNoise(p=0.3, noise_level=0.03),
])

# Tập validation không sử dụng augmentation
val_transform = None

print("Đã định nghĩa xong pipeline Augmentation mới cho Spectrogram.")


# --- CÁC HÀM MIXUP (Không sử dụng trong vòng lặp chính nhưng giữ lại) ---
def mixup_data(x, y, alpha=0.4, use_cuda=True):
    """
    Trộn dữ liệu trong một batch.
    Trả về input đã trộn, và hai bộ nhãn gốc cùng với hệ số trộn lambda.
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """
    Hàm tính loss đặc biệt cho Mixup.
    """
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

Đã định nghĩa xong pipeline Augmentation mới cho Spectrogram.


In [35]:
class DynamicGraphAttention(nn.Module):
    def __init__(self, in_channels, num_nodes=32, heads=8, dropout=0.1):
        super().__init__()
        self.num_nodes, self.heads = num_nodes, heads
        self.head_dim = in_channels // heads
        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(in_channels, in_channels)
        self.key = nn.Linear(in_channels, in_channels)
        self.value = nn.Linear(in_channels, in_channels)
        self.out = nn.Linear(in_channels, in_channels)
        self.scale = nn.Parameter(torch.tensor(self.head_dim**-0.5))
        self.pos_enc = nn.Parameter(torch.randn(1, num_nodes, in_channels))

    def forward(self, x):
        B, C, H, W = x.size()
        x_flat = x.flatten(2).transpose(1, 2)
        num_pixels = H * W
        N = min(self.num_nodes, num_pixels)
        _, indices = torch.topk(torch.norm(x_flat, dim=-1), N, dim=-1)
        indices_expanded = indices.unsqueeze(-1).expand(-1, -1, C)
        x_nodes = torch.gather(x_flat, 1, indices_expanded)
        if self.pos_enc.size(1) >= N:
            x_nodes = x_nodes + self.pos_enc[:, :N, :]
        q = self.query(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        k = self.key(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        v = self.value(x_nodes).view(B, N, self.heads, self.head_dim).transpose(1, 2)
        attn_weights = F.softmax(torch.matmul(q, k.transpose(-2, -1)) * self.scale, dim=-1)
        attn_output = torch.matmul(self.dropout(attn_weights), v).transpose(1, 2).contiguous().view(B, N, C)
        out_nodes = self.out(attn_output)
        out_flat = torch.zeros_like(x_flat)
        out_flat.scatter_(1, indices_expanded, out_nodes)
        out = out_flat.transpose(1, 2).view(B, C, H, W)
        return x + out

class TemporalAttentionAggregator(nn.Module):
    def __init__(self, in_channels, num_heads=4):
        super().__init__()
        self.attention_net = nn.MultiheadAttention(embed_dim=in_channels, num_heads=num_heads, batch_first=True)
        self.norm = nn.LayerNorm(in_channels)

    def forward(self, x):
        if x.dim() != 4: raise ValueError("Input must be 4D tensor")
        B, C, H, W = x.size()
        x_flat = x.flatten(2).transpose(1, 2)
        x_norm = self.norm(x_flat)
        attn_output, _ = self.attention_net(x_norm, x_norm, x_norm)
        return torch.mean(attn_output, dim=1)

class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expansion=6):
        super().__init__()
        hidden_dim = in_channels * expansion
        self.use_residual = in_channels == out_channels and stride == 1
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, 1, bias=False), nn.BatchNorm2d(hidden_dim), nn.SiLU(inplace=True),
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False), nn.BatchNorm2d(hidden_dim), nn.SiLU(inplace=True),
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False), nn.BatchNorm2d(out_channels)
        )
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, out_channels // 16, 1), nn.SiLU(),
            nn.Conv2d(out_channels // 16, out_channels, 1), nn.Sigmoid()
        )

    def forward(self, x):
        out = self.conv(x)
        out = out * self.se(out)
        return x + out if self.use_residual else out


In [36]:
class PestLarvaNet_Medium_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.config = [[1, 24, 2, 1], [6, 40, 3, 2], [6, 64, 3, 2], [6, 120, 3, 2], [6, 160, 2, 1]]
        self.stem = nn.Sequential(
            nn.Conv2d(1, 24, 3, 2, 1, bias=False),
            nn.BatchNorm2d(24),
            nn.SiLU(inplace=True)
        )
        layers = []
        in_channels = 24
        for t, c, n, s in self.config:
            for i in range(n):
                stride = s if i == 0 else 1
                layers.append(InvertedResidual(in_channels, c, stride, expansion=t))
                in_channels = c
            if c == 40:
                layers.append(DynamicGraphAttention(in_channels, num_nodes=32, heads=8))
        self.body = nn.Sequential(*layers)
        self.head_conv = nn.Conv2d(160, 384, 1, bias=False)
        self.aggregator = TemporalAttentionAggregator(384, num_heads=8)
    
    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(1)
        x = self.stem(x)
        x = self.body(x)
        x = self.head_conv(x)
        x = self.aggregator(x)
        return x

class PestLarvaNet_Medium_Classifier(nn.Module):
    def __init__(self, num_classes, pretrained_weights_path=None):
        super().__init__()
        self.encoder = PestLarvaNet_Medium_Encoder()
        self.classifier = nn.Sequential(
            nn.Linear(384, 192),
            nn.SiLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(192, num_classes)
        )
        if pretrained_weights_path:
            print(f"Loading pretrained weights from: {pretrained_weights_path}")
            checkpoint = torch.load(pretrained_weights_path, map_location=device, weights_only=True)
            # Adjust the checkpoint loading to handle encoder weights
            state_dict = {k.replace('encoder.', ''): v for k, v in checkpoint.items() if k.startswith('encoder.')}
            self.encoder.load_state_dict(state_dict, strict=False)

    def forward(self, x):
        x = self.encoder(x)
        return self.classifier(x)

In [37]:
class SpectrogramDataset(Dataset):
    def __init__(self, metadata_df, data_dir, transform=None, target_tdim=216): # Cập nhật tdim nếu cần
        super().__init__()
        self.metadata = metadata_df
        self.data_dir = data_dir
        self.transform = transform
        self.target_tdim = target_tdim
        # Tạo ánh xạ từ tên lớp sang ID số nguyên
        self.class_names = sorted(self.metadata['label'].unique())
        self.class_to_id = {name: i for i, name in enumerate(self.class_names)}

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        # THAY ĐỔI: Sử dụng cột 'tensor_path'
        relative_path = row['tensor_path']
        # THAY ĐỔI: Kết hợp thư mục gốc với đường dẫn tương đối
        file_path = os.path.join(self.data_dir, relative_path)
        label = self.class_to_id[row['label']]

        try:
            spectrogram = torch.load(file_path)
            # Chuẩn hóa kích thước spectrogram
            current_tdim = spectrogram.shape[2]
            if current_tdim > self.target_tdim:
                spectrogram = spectrogram[:, :, :self.target_tdim]
            elif current_tdim < self.target_tdim:
                spectrogram = F.pad(spectrogram, (0, self.target_tdim - current_tdim))

            # Áp dụng augmentation nếu có
            if self.transform:
                spectrogram = self.transform(spectrogram)
            return spectrogram, label
        except Exception as e:
            print(f"Error loading or processing file {file_path}: {e}")
            return None

def collate_fn_remove_none(batch):
    batch = [item for item in batch if item is not None]
    if not batch: return None, None
    samples, labels = zip(*batch)
    return torch.stack(samples), torch.tensor(labels)

In [38]:
def train_and_evaluate(model, train_loader, val_loader, epochs, checkpoint_path, metrics_csv_path):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.5, patience=5, verbose=False)
    # Dùng GradScaler mà không cần kiểm tra device.type
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
    best_f1 = 0.0
    
    if not os.path.exists(metrics_csv_path):
        pd.DataFrame(columns=['Epoch', 'Accuracy', 'Precision', 'Recall', 'F1-Score']).to_csv(metrics_csv_path, index=False)
        
    for epoch in range(epochs):
        model.train()
        # Bọc train_loader bằng tqdm để có thanh tiến trình
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for inputs, labels in loop:
            if inputs is None: continue
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad(set_to_none=True) # Tối ưu hóa
            
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                if inputs is None: continue
                inputs, labels = inputs.to(device), labels.to(device)
                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                    outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                
        if len(val_labels) > 0:
            current_f1 = f1_score(val_labels, val_preds, average='weighted', zero_division=0)
            if current_f1 > best_f1:
                best_f1 = current_f1
                current_accuracy = accuracy_score(val_labels, val_preds)
                current_precision = precision_score(val_labels, val_preds, average='weighted', zero_division=0)
                current_recall = recall_score(val_labels, val_preds, average='weighted', zero_division=0)
                
                print(f"\nEpoch {epoch+1}: New best F1-score: {best_f1:.4f}")
                print(f"Accuracy: {current_accuracy:.4f}, Precision: {current_precision:.4f}, Recall: {current_recall:.4f}")
                print(f"Saving checkpoint to {checkpoint_path}")
                torch.save(model.state_dict(), checkpoint_path)
                
                metrics_df = pd.DataFrame([{'Epoch': epoch + 1, 'Accuracy': current_accuracy, 'Precision': current_precision, 'Recall': current_recall, 'F1-Score': current_f1}])
                metrics_df.to_csv(metrics_csv_path, mode='a', header=False, index=False)
            scheduler.step(current_f1)
            
    print(f"Finished Training for this run. Best F1-score: {best_f1:.4f}")
    # <<< SỬA LỖI: Chỉ trả về một giá trị float duy nhất >>>
    return best_f1

In [39]:
# --- CẬP NHẬT CÁC ĐƯỜNG DẪN NÀY ---
ROOT_DIR = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium'
METADATA_PATH = os.path.join(ROOT_DIR, 'spectrogram_tensor_metadata.csv')
CHECKPOINT_DIR = os.path.join(ROOT_DIR, 'training_checkpoints')
CHECKPOINT_BASE = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

checkpoint_files = [
    'run_1_lp1e-04_ep1e-06_lb0.001_audiotwin_pestlarvanet_medium_epoch_100.pth',
    'run_2_lp5e-04_ep1e-06_lb0.003_audiotwin_pestlarvanet_medium_epoch_100.pth',
    'run_3_lp1e-03_ep1e-06_lb0.005_audiotwin_pestlarvanet_medium_epoch_100.pth',
    'run_4_lp5e-03_ep1e-06_lb0.007_audiotwin_pestlarvanet_medium_epoch_100.pth',
    'run_5_lp1e-02_ep1e-06_lb0.010_audiotwin_pestlarvanet_medium_epoch_100.pth'
]
DATA_PERCENTAGES = [0.25, 0.5, 1.0]
results = {}

# 1. Tải và chuẩn bị dữ liệu (thực hiện một lần)
print("Preparing dataset...")
full_metadata = pd.read_csv(METADATA_PATH)
num_classes = len(full_metadata['label'].unique())
print(f"Number of classes: {num_classes}")

indices = np.arange(len(full_metadata))
labels = full_metadata['label'].values
train_indices, test_indices, _, _ = train_test_split(
    indices, labels, test_size=0.2, random_state=SEED, stratify=labels
)
print(f"Data split: {len(train_indices)} train, {len(test_indices)} test.")

# Tạo dataset test cố định
test_metadata = full_metadata.iloc[test_indices]
test_dataset = SpectrogramDataset(metadata_df=test_metadata, data_dir=ROOT_DIR, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=0, collate_fn=collate_fn_remove_none)

# 2. Xây dựng cấu hình mô hình
model_configs = {
    f"PestLarvaNet (AudioTwin Run {i+1})": {
        "model_class": PestLarvaNet_Medium_Classifier,
        "args": {"num_classes": num_classes, "pretrained_weights_path": os.path.join(CHECKPOINT_BASE, checkpoint_files[i])}
    } for i in range(len(checkpoint_files))
}

Preparing dataset...
Number of classes: 8
Data split: 7316 train, 1830 test.


In [40]:
for model_name, config in model_configs.items():
    print(f"\n{'='*20} Evaluating: {model_name} {'='*20}")
    model_results = []
    for percent in DATA_PERCENTAGES:
        print(f"--- Training with {percent*100:.1f}% of labeled data ---")

        if percent < 1.0:
            stratify_labels = full_metadata['label'].iloc[train_indices].values
            subset_train_indices, _ = train_test_split(
                train_indices, train_size=percent, random_state=SEED, stratify=stratify_labels
            )
        else:
            subset_train_indices = train_indices

        train_metadata_subset = full_metadata.iloc[subset_train_indices]
        train_dataset = SpectrogramDataset(metadata_df=train_metadata_subset, data_dir=ROOT_DIR, transform=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, collate_fn=collate_fn_remove_none)
        
        checkpoint_filename = f"{model_name.replace(' ', '_').replace('(', '').replace(')', '')}_data_{percent*100:.1f}percent.pth"
        checkpoint_path = os.path.join(CHECKPOINT_DIR, checkpoint_filename)
        metrics_csv_path = os.path.join(CHECKPOINT_DIR, f"{model_name.replace(' ', '_').replace('(', '').replace(')', '')}_data_{percent*100:.1f}percent_metrics.csv")

        model = config["model_class"](**config["args"]).to(device)
        
        f1 = train_and_evaluate(model, train_loader, test_loader, epochs=50, checkpoint_path=checkpoint_path, metrics_csv_path=metrics_csv_path)
        model_results.append({'percent': percent, 'f1': f1})
        print(f"==> Results with {percent*100:.1f}% data: Best F1-Score = {f1:.4f}")
    
    results[model_name] = model_results

print("\n--- FINAL RESULTS ---")
for model_name, result_list in results.items():
    print(f"\nModel: {model_name}")
    for res in result_list:
        print(f"  {res['percent']*100:.1f}% data: F1-Score = {res['f1']:.4f}")


==================== Evaluating: PestLarvaNet (AudioTwin Run 1) ====================
--- Training with 25.0% of labeled data ---
Loading pretrained weights from: C:\Users\PC\lavar\audiotwin_checkpoints_medium\run_1_lp1e-04_ep1e-06_lb0.001_audiotwin_pestlarvanet_medium_epoch_100.pth


C:\Users\PC\anaconda3\envs\train_env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50:   0%|          | 0/58 [00:00<?, ?it/s]


Epoch 1: New best F1-score: 0.3518
Accuracy: 0.4858, Precision: 0.4682, Recall: 0.4858
Saving checkpoint to C:\Users\PC\lavar\audiotwin_checkpoints_medium\training_checkpoints\PestLarvaNet_AudioTwin_Run_1_data_25.0percent.pth


Epoch 2/50:   0%|          | 0/58 [00:00<?, ?it/s]

...

Epoch 48/50:   0%|          | 0/229 [00:00<?, ?it/s]


Epoch 48: New best F1-score: 0.8915
Accuracy: 0.8929, Precision: 0.8937, Recall: 0.8929
Saving checkpoint to C:\Users\PC\lavar\audiotwin_checkpoints_medium\training_checkpoints\PestLarvaNet_AudioTwin_Run_5_data_100.0percent.pth


Epoch 49/50:   0%|          | 0/229 [00:00<?, ?it/s]

Epoch 50/50:   0%|          | 0/229 [00:00<?, ?it/s]


Epoch 50: New best F1-score: 0.8962
Accuracy: 0.8973, Precision: 0.8979, Recall: 0.8973
Saving checkpoint to C:\Users\PC\lavar\audiotwin_checkpoints_medium\training_checkpoints\PestLarvaNet_AudioTwin_Run_5_data_100.0percent.pth
Finished Training for this run. Best F1-score: 0.8962
==> Results with 100.0% data: Best F1-Score = 0.8962

--- FINAL RESULTS ---

Model: PestLarvaNet (AudioTwin Run 1)
  25.0% data: F1-Score = 0.8495
  50.0% data: F1-Score = 0.8971
  100.0% data: F1-Score = 0.9178

Model: PestLarvaNet (AudioTwin Run 2)
  25.0% data: F1-Score = 0.7889
  50.0% data: F1-Score = 0.8946
  100.0% data: F1-Score = 0.9237

Model: PestLarvaNet (AudioTwin Run 3)
  25.0% data: F1-Score = 0.7224
  50.0% data: F1-Score = 0.8680
  100.0% data: F1-Score = 0.9200

Model: PestLarvaNet (AudioTwin Run 4)
  25.0% data: F1-Score = 0.7987
  50.0% data: F1-Score = 0.8921
  100.0% data: F1-Score = 0.8820

Model: PestLarvaNet (AudioTwin Run 5)
  25.0% data: F1-Score = 0.8330
  50.0% data: F1-Score = 0